# <center>使用CI方法求解一维ising模型精确基态能量</center>

<div align=center>
<img src="图片\2.jpg" width="1000" height="1000" />
    </div >

**<center>一维ising模型的Hamitonian已知</center>**
$$\hat{H} = -J\sum_{i}s_is_{i+1} - B\sum_i s_i $$

 例如：当格点数为 4 时， Hamitonian可以写成：
 $$H = -\sigma_1^z\otimes \sigma_2^z \otimes I_3 \otimes I_4 - I_1\otimes \sigma_2^z \otimes \sigma_3^z \otimes I_4 - I_1 \otimes I_2 \otimes \sigma_3^z \otimes \sigma_4^z - \sigma_1^x \otimes I_2\otimes I_3\otimes I_4 - I_1\otimes \sigma_2^x \otimes I_3\otimes I_4 - I_1\otimes I_2\otimes \sigma_3^x \otimes I_4 - I_1\otimes I_2\otimes I_3\otimes \sigma_4^x $$

**我们的任务**：把H写出来，然后写出H在自旋表象下 (|0000> ,|0001>,...,|1111> 16个基下)的H矩阵,对角化H矩阵即可得到基态能量。

In [1]:
import numpy as np
import time
# Sx measurement
Sx = np.float64([[0,   0.5],
                    [0.5, 0  ]])

# Sz measurement
Sz = np.float64([[0.5,  0   ],
                    [0,   -0.5]])

## 写出自旋表象下的H矩阵
$$\hat{H} = -J\sum_{i}\sigma_i^z\sigma_{i+1}^z - B\sum_i \sigma_i^x $$

In [2]:
# 生成L个格点的H矩阵， 默认J = B = 1， 因为我们可以直接写出自旋表象下的H矩阵，所以没有使用<000|H|000>这种写法，效果是一样的。
def Ham(L): 
    I = np.eye(2)
    e = 2**L
    J = np.zeros((e,e))
    #print(J)
    B = np.zeros((e,e))
    
    #计算相互作用矩阵，注意维度为2**L, 要张量积其他格点单位矩阵
    for i in range(0, L-1):
        _Jz = 1
        for j in range(0, i):
            _Jz = np.kron(_Jz, I)
        if(i > L-2):
            _Jz = np.kron(_Jz,-Sz)
        else:
            _Jz = np.kron(np.kron(_Jz,-Sz),Sz)
        for k in range(i+2, L):
            _Jz = np.kron(_Jz, I)
            #print("**JZ")
            #print(_Jz)
        J += _Jz
        #print(J)
              
    #计算外场作用的矩阵，也要张量积其他格点单位矩阵
    for i in range(0, L):
        _Bx = 1
        for j in range(0, i):
            _Bx = np.kron(_Bx, I)
        _Bx = np.kron(_Bx,Sx)
        for j in range(i+1, L):
            _Bx = np.kron(_Bx, I)
        B += _Bx     
    return J-B
        
    
    


In [3]:
def run(L):
    H = Ham(L)
    print("H矩阵为：\n",H)
    eigVal, eigVec = np.linalg.eigh(H)
    return eigVal

## 测试N = 4

In [5]:
t0 = time.process_time()
out = run(4)
t1 = time.process_time()
print("使用CI计算所用时间为：",t1 - t0)
print("计算得到精确解为：",out[0])

H矩阵为：
 [[-0.75 -0.5  -0.5   0.   -0.5   0.    0.    0.   -0.5   0.    0.    0.
   0.    0.    0.    0.  ]
 [-0.5  -0.25  0.   -0.5   0.   -0.5   0.    0.    0.   -0.5   0.    0.
   0.    0.    0.    0.  ]
 [-0.5   0.    0.25 -0.5   0.    0.   -0.5   0.    0.    0.   -0.5   0.
   0.    0.    0.    0.  ]
 [ 0.   -0.5  -0.5  -0.25  0.    0.    0.   -0.5   0.    0.    0.   -0.5
   0.    0.    0.    0.  ]
 [-0.5   0.    0.    0.    0.25 -0.5  -0.5   0.    0.    0.    0.    0.
  -0.5   0.    0.    0.  ]
 [ 0.   -0.5   0.    0.   -0.5   0.75  0.   -0.5   0.    0.    0.    0.
   0.   -0.5   0.    0.  ]
 [ 0.    0.   -0.5   0.   -0.5   0.    0.25 -0.5   0.    0.    0.    0.
   0.    0.   -0.5   0.  ]
 [ 0.    0.    0.   -0.5   0.   -0.5  -0.5  -0.25  0.    0.    0.    0.
   0.    0.    0.   -0.5 ]
 [-0.5   0.    0.    0.    0.    0.    0.    0.   -0.25 -0.5  -0.5   0.
  -0.5   0.    0.    0.  ]
 [ 0.   -0.5   0.    0.    0.    0.    0.    0.   -0.5   0.25  0.   -0.5
   0.   -0.5   0.    0.  ]
 

## 测试N = 10

In [19]:
t0 = time.process_time()
out = run(10)
t1 = time.process_time()
print("使用CI计算所用时间为：",t1 - t0)
print("计算得到精确解为：",out[1])

H矩阵为：
 [[-2.25 -0.5  -0.5  ...  0.    0.    0.  ]
 [-0.5  -1.75  0.   ...  0.    0.    0.  ]
 [-0.5   0.   -1.25 ...  0.    0.    0.  ]
 ...
 [ 0.    0.    0.   ... -1.25  0.   -0.5 ]
 [ 0.    0.    0.   ...  0.   -1.75 -0.5 ]
 [ 0.    0.    0.   ... -0.5  -0.5  -2.25]]
使用CI计算所用时间为： 1.5650366479999995
计算得到精确解为： -4.7515029807284135


## 测试N = 15, （内存所需过大，可能会报错，我的运行结果见下图）

In [7]:
#t0 = time.process_time()
#out = run(15)
#t1 = time.process_time()
#print("使用CI计算所用时间为：",t1 - t0)
#print("计算得到精确解为：",out[1])

<div align=center>
<img src="图片\36.jpg" width="1000" height="1000" />
    </div >

## 可见传统CI方法在模型规模稍大一些，就因为存储问题难以进行计算了。
## 而DMRG方法因为截断过程，相对减少了存储开支和计算量。